In [30]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import re
import string
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import nltk
os.makedirs(r'C:\\nltk_data', exist_ok=True)
nltk.data.path.append(r'C:\\nltk_data')  
nltk.download('punkt_tab', download_dir=r'C:\\nltk_data')
nltk.download('stopwords', download_dir=r'C:\\nltk_data')
nltk.download('wordnet', download_dir=r'C:\\nltk_data')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from tqdm import tqdm

[nltk_data] Downloading package punkt_tab to C:\\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
import pandas as pd

def read_database_result_csv(file_path):
    try:
        # Mencoba membaca dengan encoding default (utf-8) dan delimiter koma
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    except UnicodeDecodeError as e1:
        try:
            # Jika gagal, mencoba dengan encoding latin1
            df = pd.read_csv(file_path, sep=';', encoding='latin1')
        except UnicodeDecodeError as e2:
            try:
                # Jika gagal, mencoba dengan encoding cp1252
                df = pd.read_csv(file_path, sep=';', encoding='cp1252')
            except Exception as e3:
                print(f"Gagal membaca file {file_path}: {e1} | {e2} | {e3}")
                return pd.DataFrame()  # Mengembalikan DataFrame kosong jika gagal
    return df

# Path ke file DATABASE_RESULT_OJK.csv
file_path = 'data/IDI.csv'
database_result_df = read_database_result_csv(file_path)

print(database_result_df.head())


     Responden     Jenis                                   Kualitas Layanan  \
0  Responden 1  Asosiasi  OJK memiliki layanan syariah yang mana semua p...   
1  Responden 2  Asosiasi  asosiasi sangat puas atas layanan dan dukungan...   
2  Responden 3  Asosiasi  Meski perlu ada peningkatan koordinasi dan kom...   
3  Responden 4  Asosiasi                                   Sudah cukup baik   
4  Responden 5  Asosiasi                          Masih perlu ditingkatkan.   

                                 Hubungan dengan OJK  \
0  Hubungan yang ada mampu menjadi dorongan penti...   
1  Setelah pendirian ITSK proses pendirian perusa...   
2  SPRINT sudah mumpuni dan memberikan pelayanan ...   
3  inovasi layanan masih perlu disediakan untuk m...   
4                                        Sandbox 2.0   

                                        Kualitas SDM  \
0  SDM di OJK sudah kompeten dan memiliki kemampu...   
1  Kompetensi pegawai OJK sudah sangat bagus dan ...   
2       OJK sudah ca

In [32]:
# #FILTERING SATKER

# # Menentukan filter SATKER yang diinginkan
# filter_option = input("Apakah Anda ingin memilih SATKER berdasarkan pilihan tertentu (y/n)? ")

# if filter_option.lower() == 'y':
#     # Jika pengguna memilih untuk memasukkan SATKER
#     selected_satker = input("Masukkan SATKER yang ingin difilter (misal: 'DPEA'): ")
    
#     # Memfilter data berdasarkan SATKER yang dipilih
#     filtered_df = database_result_df[database_result_df['SATKER (AKRONIM)'] == selected_satker]
#     print(f"Data difilter berdasarkan SATKER: {selected_satker}")
    
# elif filter_option.lower() == 'n':
#     # Jika pengguna memilih untuk memfilter berdasarkan beberapa SATKER tertentu
#     selected_satker_list = ['DPEA', 'DOSB', 'DPSI']  # Contoh daftar SATKER yang bisa dipilih
#     filtered_df = database_result_df[database_result_df['SATKER (AKRONIM)'].isin(selected_satker_list)]
#     print(f"Data difilter berdasarkan beberapa SATKER: {', '.join(selected_satker_list)}")
    
# else:
#     # Jika input selain 'y' atau 'n', tampilkan pesan
#     print("Pilihan tidak valid. Menggunakan data tanpa filter.")
#     filtered_df = database_result_df

# # Menampilkan data yang sudah difilter
# print(filtered_df)

In [33]:
# print(database_result_df.isnull().sum())

In [34]:
columns_order = [
    'Responden', 'Jenis', 'Kualitas Layanan', 'Hubungan dengan OJK', 'Kualitas SDM', 'Saran'
]

In [35]:
all_data_idi_df = database_result_df[columns_order]

In [36]:
print(all_data_idi_df)

                            Responden      Jenis  \
0                         Responden 1   Asosiasi   
1                         Responden 2   Asosiasi   
2                         Responden 3   Asosiasi   
3                         Responden 4   Asosiasi   
4                         Responden 5   Asosiasi   
5                         Responden 6   Asosiasi   
6                         Responden 7   Asosiasi   
7                         Responden 8   Asosiasi   
8                         Responden 9   Asosiasi   
9             Responden 10 (Dr. Muiz)  Akademisi   
10            Responden 11(Dr. Wendy)  Akademisi   
11      Responden 12(Prof. Irwan Adi)  Akademisi   
12  Responden 13(Prof. Mamduh Hanafi)  Akademisi   

                                     Kualitas Layanan  \
0   OJK memiliki layanan syariah yang mana semua p...   
1   asosiasi sangat puas atas layanan dan dukungan...   
2   Meski perlu ada peningkatan koordinasi dan kom...   
3                                    Sudah 

In [37]:
print(all_data_idi_df.isnull().sum())

Responden              0
Jenis                  0
Kualitas Layanan       0
Hubungan dengan OJK    0
Kualitas SDM           0
Saran                  0
dtype: int64


In [38]:
all_data_idi_df.describe()

,Responden,Jenis,Kualitas Layanan,Hubungan dengan OJK,Kualitas SDM,Saran
count,13,13,13,13,13,13
unique,13,2,9,13,13,12
top,Responden 1,Asosiasi,cukup baik,Hubungan yang ada mampu menjadi dorongan penti...,SDM di OJK sudah kompeten dan memiliki kemampu...,-
freq,1,9,3,1,1,2


In [39]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+|\@\w+|\#|\d+|[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    stop_words = set(stopwords.words('indonesian'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    cleaned_text = ' '.join(words)
    
    return cleaned_text

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

# Ganti model_name dengan model IndoBERT
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=6  # Sesuaikan jumlah label
)

label_map = {
    0: "sangat tidak setuju",
    1: "tidak setuju",
    2: "kurang setuju",
    3: "cukup setuju",
    4: "setuju",
    5: "sangat setuju"
}

# Fungsi untuk memproses teks
def preprocess_text(text):
    return text.strip().lower() if isinstance(text, str) else ""  # Periksa apakah teks valid

# Fungsi untuk prediksi sentimen
def predict_sentiment(texts):
    model.eval()
    results = []
    
    with torch.no_grad():
        for text in tqdm(texts):
            text = preprocess_text(text)
            
            if not text:  # Lewati teks kosong
                results.append({
                    'text': text,
                    'sentiment': "unknown",
                    'confidence': 0.0
                })
                continue
            
            # Tokenisasi teks
            encoded = tokenizer(
                text,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            )
            
            # Model memprediksi
            outputs = model(encoded["input_ids"], attention_mask=encoded["attention_mask"])
            predictions = F.softmax(outputs.logits, dim=1)
            predicted_label = torch.argmax(predictions, dim=1).item()
            
            # Ambil confidence score
            confidence = predictions[0][predicted_label].item()
            
            # Mapping hasil prediksi ke label
            sentiment = label_map.get(predicted_label, "unknown")
            
            # Menambahkan hasil ke dalam list
            results.append({
                'text': text,
                'sentiment': sentiment,
                'confidence': confidence
            })
    
    return pd.DataFrame(results)

# Gabungkan teks dari kolom-kolom yang diinginkan
columns_to_process = ['Kualitas Layanan', 'Hubungan dengan OJK', 'Kualitas SDM', 'Saran']
all_data_idi_df['Combined_Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)

# Memanggil fungsi prediksi
results = predict_sentiment(all_data_idi_df['Combined_Text'].tolist())

# Menambahkan hasil prediksi ke dataframe asli menggunakan .loc
all_data_idi_df.loc[:, 'Label'] = results['sentiment']
all_data_idi_df.loc[:, 'Confidence'] = results['confidence']

# Menampilkan hasil
print("\nSample of labeled data:")
print(all_data_idi_df[['Kualitas Layanan', 'Hubungan dengan OJK', 'Kualitas SDM', 'Saran', 'Label', 'Confidence']].head())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 13/13 [00:07<00:00,  1.65it/s]


Sample of labeled data:
                                    Kualitas Layanan  \
0  OJK memiliki layanan syariah yang mana semua p...   
1  asosiasi sangat puas atas layanan dan dukungan...   
2  Meski perlu ada peningkatan koordinasi dan kom...   
3                                   Sudah cukup baik   
4                          Masih perlu ditingkatkan.   

                                 Hubungan dengan OJK  \
0  Hubungan yang ada mampu menjadi dorongan penti...   
1  Setelah pendirian ITSK proses pendirian perusa...   
2  SPRINT sudah mumpuni dan memberikan pelayanan ...   
3  inovasi layanan masih perlu disediakan untuk m...   
4                                        Sandbox 2.0   

                                        Kualitas SDM  \
0  SDM di OJK sudah kompeten dan memiliki kemampu...   
1  Kompetensi pegawai OJK sudah sangat bagus dan ...   
2       OJK sudah canggih dengan SDM yang sudah baik   
3  yang masih menjadi masalah adalah struktur org...   
4                    

In [41]:
all_data_idi_df.to_csv('data/hasil/main_data_idi.csv', index=False, sep=';')

In [42]:
print(all_data_idi_df)

                            Responden      Jenis  \
0                         Responden 1   Asosiasi   
1                         Responden 2   Asosiasi   
2                         Responden 3   Asosiasi   
3                         Responden 4   Asosiasi   
4                         Responden 5   Asosiasi   
5                         Responden 6   Asosiasi   
6                         Responden 7   Asosiasi   
7                         Responden 8   Asosiasi   
8                         Responden 9   Asosiasi   
9             Responden 10 (Dr. Muiz)  Akademisi   
10            Responden 11(Dr. Wendy)  Akademisi   
11      Responden 12(Prof. Irwan Adi)  Akademisi   
12  Responden 13(Prof. Mamduh Hanafi)  Akademisi   

                                     Kualitas Layanan  \
0   OJK memiliki layanan syariah yang mana semua p...   
1   asosiasi sangat puas atas layanan dan dukungan...   
2   Meski perlu ada peningkatan koordinasi dan kom...   
3                                    Sudah 

In [43]:
# # Menyimpan data ke file CSV
# output_file = "./data/hasil/all_data_idi_ss.csv"
# all_data_idi_df.to_csv(output_file, index=False, sep=';', encoding='utf-8-sig')

# print(f"Data berhasil disimpan ke {output_file}")

In [44]:
#SAVE MODEL

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Ganti model_name dengan model IndoBERT
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=6  # Sesuaikan jumlah label
)

# Menyimpan model dan tokenizer ke direktori
model_save_path = './saved_model'
tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f"Model dan tokenizer berhasil disimpan di {model_save_path}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model dan tokenizer berhasil disimpan di ./saved_model


In [45]:
#LOAD MODEL

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Path ke model yang disimpan
model_save_path = './saved_model'

# Memuat model dan tokenizer dari path yang telah disimpan
tokenizer = AutoTokenizer.from_pretrained(model_save_path)
model = AutoModelForSequenceClassification.from_pretrained(model_save_path)

print(f"Model dan tokenizer berhasil dimuat dari {model_save_path}")

Model dan tokenizer berhasil dimuat dari ./saved_model


In [46]:
label_counts = all_data_idi_df['Label'].value_counts()

label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']
print(label_summary)

                 Label  Count
0         cukup setuju     10
1  sangat tidak setuju      3


In [47]:
# os.makedirs('data/hasil', exist_ok=True)
# all_data_idi_df.to_csv('data/hasil/labeled.csv', index=False, sep=";")

In [48]:
import json
#VALIDASI

file_path = 'data/hasil/main_data_idi.csv'

# Load dataset
data = pd.read_csv(file_path, delimiter=';')

data_copy = data.copy()

# Membaca file JSON yang berisi mapping
with open('mapping.json', 'r') as file:
    text_mapping = json.load(file)


# Fungsi untuk menentukan label berdasarkan kata-kata dalam 'Combined_Text'
def map_combined_text_to_label(text):
    # Iterasi untuk mencari kata kunci dalam text
    for keyword, label in text_mapping.items():
        if keyword in text.lower():  # Mengabaikan kapitalisasi
            return label
    return 'setuju'  # Jika tidak ada kata kunci yang ditemukan

# Terapkan fungsi ke kolom 'Combined_Text' untuk membuat kolom 'Label'
data_copy['New_Label'] = data_copy['Combined_Text'].apply(map_combined_text_to_label)

# Tampilkan dataset yang sudah dimodifikasi
data_copy['Label'] = data_copy['New_Label'].combine_first(data_copy['Label'])

# Simpan dataset yang sudah dimodifikasi
# data_copy.to_csv('data/hasil/validated_dataset_new.csv', index=False, sep=';')

# Menampilkan statistik label setelah perubahan
# label_counts = data_copy['Label'].value_counts()
# label_summary = pd.DataFrame(label_counts).reset_index()
# label_summary.columns = ['Label', 'Count']

label_map = {
    1: "sangat tidak setuju",
    2: "tidak setuju",
    3: "kurang setuju",
    4: "cukup setuju",
    5: "setuju",
    6: "sangat setuju"
}

reverse_label_map = {v: k for k, v in label_map.items()}  

data_copy['Label_Index'] = data_copy['Label'].map(reverse_label_map)

def calculate_weight(index):
    if index == 1:
        return 1
    elif 1.1 <= index <= 2.9:
        return 2.95
    elif 3 <= index <= 3.9:
        return 5.9
    elif 4 <= index <= 6:
        return 6
    else:
        return None  

data_copy['NILAI_SENTIMEN'] = data_copy['Label_Index'].apply(calculate_weight)

# print(data_copy)


print(data_copy[['Label','NILAI_SENTIMEN']])
data_copy.to_csv('data/hasil/main_data_idi.csv', index=False, sep=';')

label_counts = data_copy['Label'].value_counts()
label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']

     Label  NILAI_SENTIMEN
0   setuju               6
1   setuju               6
2   setuju               6
3   setuju               6
4   setuju               6
5   setuju               6
6   setuju               6
7   setuju               6
8   setuju               6
9   setuju               6
10  setuju               6
11  setuju               6
12  setuju               6


In [49]:
# import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer
# from nltk.corpus import stopwords
# import nltk

# # Pastikan Anda sudah mengunduh stopwords dari NLTK
# nltk.download('stopwords')

# # Fungsi untuk membersihkan dan menghilangkan stopwords
# def clean_text(text):
#     # Ubah ke huruf kecil dan tokenisasi
#     text = text.lower()
#     # Tokenisasi kata
#     words = text.split()
#     # Menghilangkan stopwords menggunakan NLTK
#     stop_words = set(stopwords.words('indonesian'))  # Sesuaikan dengan bahasa dataset
#     words = [word for word in words if word not in stop_words]
#     return ' '.join(words)

# # Asumsikan `all_data_idi_df` sudah memiliki kolom `Combined_Text`
# # Gabungkan teks dari kolom yang relevan jika belum ada
# columns_to_process = ['NAMA PIC/RESPONDEN', 'OPEN QUESTION 1', 'OPEN QUESTION 2']
# all_data_idi_df['Combined_Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)

# # Bersihkan teks dari stopwords
# all_data_idi_df['Cleaned_Text'] = all_data_idi_df['Combined_Text'].apply(clean_text)

# # Inisialisasi CountVectorizer untuk menghitung frekuensi kata
# vectorizer = CountVectorizer(max_features=1000, ngram_range=(1, 1))  # Menggunakan unigrams (kata tunggal)
# X = vectorizer.fit_transform(all_data_idi_df['Cleaned_Text'])

# # Mendapatkan kata-kata dan frekuensinya
# word_counts = X.toarray().sum(axis=0)
# words = vectorizer.get_feature_names_out()

# # Membuat dataframe untuk menampilkan hasil
# word_freq_df = pd.DataFrame(zip(words, word_counts), columns=['Word', 'Frequency'])
# word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False)

# # Menampilkan kata-kata kunci yang paling sering muncul
# print(word_freq_df.head(10))  # Menampilkan 10 kata teratas


In [50]:
# os.makedirs('data/hasil', exist_ok=True)
# all_data_idi_df.to_csv('data/hasil/cleaned.csv', index=False, sep=";")

In [51]:
# data_clean_df = pd.read_csv('data/hasil/cleaned.csv', sep=';')
# print(data_clean_df.head())

In [52]:
# print("Jumlah nilai NaN:", data_clean_df['Cleaned_Text'].isna().sum())

In [53]:
# data_clean_df = data_clean_df.dropna(subset=['Cleaned_Text'])

In [54]:
# print("Jumlah nilai NaN:", data_clean_df['Cleaned_Text'].isna().sum())

In [55]:
# X = data_clean_df['Cleaned_Text']
# y = data_clean_df['Label']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# vectorizer = TfidfVectorizer()
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

In [56]:
# model = SVC(kernel='linear') 
# model.fit(X_train_tfidf, y_train)

In [57]:
# y_pred = model.predict(X_test_tfidf)

# print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
# print(classification_report(y_test, y_pred))


In [58]:
# param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
# grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
# grid.fit(X_train_tfidf, y_train)

# print(grid.best_estimator_)

In [59]:
# new_data = ["Infrastruktur OJK sudah sangat bagus", "Saya sudah cukup puas dengan kinerja OJK", "OJK sangat bagus"]
# new_data_tfidf = vectorizer.transform(new_data)
# prediction = model.predict(new_data_tfidf)
# print(prediction)